In [1]:
from eda_imports import *

In [2]:
import pysam

In [3]:
print(datetime.datetime.today().date())

2018-07-25


In [4]:
%time adf = pd.read_csv('../../benchmark_transcriptome/UHRC1/polyA.KLEAT', sep='\t')

CPU times: user 169 ms, sys: 33.9 ms, total: 203 ms
Wall time: 214 ms


In [5]:
adf.columns

Index(['gene', 'transcript', 'transcript_strand', 'coding', 'contig', 'chromosome', 'cleavage_site', 'within_UTR', 'distance_from_annotated_site', 'ESTs', 'length_of_tail_in_contig', 'number_of_tail_reads', 'number_of_bridge_reads', 'max_bridge_read_tail_length', 'bridge_read_identities',
       'tail+bridge_reads', 'number_of_link_pairs', 'max_link_pair_length', 'link_pair_identities', 'hexamer_loc+id', '3UTR_start_end', 'flag'],
      dtype='object')

In [6]:
adf.shape

(55511, 22)

# Redo hexamer search

In [7]:
wanted_chroms = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8',
       'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22',
       'chrX', 'chrY', 'chrM']

wanted_cols = [
    'gene',
    'transcript_strand',
    'chromosome',
    'cleavage_site',
    'length_of_tail_in_contig',
    'number_of_bridge_reads',
    'max_bridge_read_tail_length',
]

In [8]:
adf2 = adf[wanted_cols].rename(columns={
    'gene': 'gene_name',
    'cleavage_site': 'clv', 
    'transcript': 'transcript_id',
    'transcript_strand': 'strand',
    'chromosome': 'seqname'
})
adf2 = adf2.query(f'seqname in {wanted_chroms}').copy()

In [9]:
# convert 1-based to 0-based
adf2['clv0'] = adf2['clv'] - 1

In [10]:
REF_FA = './tasrkleat-static/off-cloud/reference_data/Homo_sapiens.GRCh37.75.dna_sm.primary_assembly.fa'
refseq = pysam.FastaFile(REF_FA)

In [11]:
clv0s_df = adf2[['seqname', 'gene_name', 'clv0', 'strand']].drop_duplicates()

In [12]:
clv0s_df.shape

(54917, 4)

In [13]:
import search_hexamer

def search_hexamer_wrapper(refseq, chrm, clv, strand, window=50):
    # for consistency with ensembl annotation
    chrm = chrm.replace('chr', '').replace('M', 'MT')
    res = search_hexamer.search(refseq, chrm, clv, strand, window)
    if res is None:
        res = ['NA', -1, -1]
    return pd.Series(res, index=['hexamer', 'hexamer_id', 'hexamer_loc0'])

In [14]:
%%time
hexm_df = clv0s_df.apply(
    lambda row: search_hexamer_wrapper(refseq, row.seqname, row.clv0, row.strand), 
    axis=1)

CPU times: user 30.5 s, sys: 1.04 s, total: 31.6 s
Wall time: 31.6 s


In [15]:
# convert to 1-based
%time hexm_df['hexamer_loc'] = hexm_df.hexamer_loc0.apply(lambda v: v + 1 if v != -1 else v)

CPU times: user 39.1 ms, sys: 999 µs, total: 40.1 ms
Wall time: 38.6 ms


In [16]:
hexm_df.head(2)

,hexamer,hexamer_id,hexamer_loc0,hexamer_loc
0,AATAAA,1,100159911,100159912
1,ATTAAA,2,1001705,1001706


In [17]:
hexm_df.shape

(54917, 4)

In [18]:
hexm_ndf = pd.concat([clv0s_df, hexm_df], axis=1)

In [19]:
hexm_ndf.shape

(54917, 8)

In [23]:
adf2.shape

(54917, 8)

In [20]:
adf3 = adf2.merge(hexm_ndf, on=['seqname', 'gene_name', 'clv0', 'strand'], how='left')

In [25]:
assert adf3.shape[0] == adf2.shape[0]

In [26]:
adf4 = adf3.drop(['clv0', 'hexamer_loc0'], axis=1)

In [27]:
adf4.head(2)

,gene_name,strand,seqname,clv,length_of_tail_in_contig,number_of_bridge_reads,max_bridge_read_tail_length,hexamer,hexamer_id,hexamer_loc
0,PALMD,+,chr1,100159930,0,1,8,AATAAA,1,100159912
1,RP11-465B22.3,+,chr1,1001727,0,4,3,ATTAAA,2,1001706


In [28]:
%time adf3.to_pickle('../../benchmark_transcriptome/UHRC1/polyA.KLEAT.hexamer-researched.pkl')

CPU times: user 12.4 ms, sys: 10.2 ms, total: 22.5 ms
Wall time: 60.1 ms
